In [ ]:
from gensim.models import KeyedVectors

In [ ]:
we = KeyedVectors.load_word2vec_format('fasttext-sbwc.100k.vec', limit=100000)

In [ ]:
we['gato'][:50]

In [ ]:
we.most_similar(positive=['rey','mujer'], negative=['hombre'])[:3]

In [ ]:
we.most_similar(positive=['actor','mujer'], negative=['hombre'])[:3]

In [ ]:
we.most_similar(positive=['mujer','activista'])

In [ ]:
# Los anteriores ejemplos muestran esta relación

# 'rey' + 'mujer' - 'hombre'
# 'actor' + 'mujer' - 'hombre'

# Puedes probar tu las siguientes relaciones

# 'dilma' + 'chile' - 'bachelet'
# 'dilma' + 'chile' - 'brasil'
# 'chile' + 'brasil'
# 'microsoft'
# 'micorosft' + 'steve'
# 'facebook'
# 'facebook' + 'bill'

# Conjugaciones

# 'corría' + 'saltar' - 'correr'
# 'corría' + 'ir' - 'correr'

# Palabras complejas

# 'mujer' + 'yerno' - 'hombre'
# 'hombre' + 'nuera' - 'mujer'

# no calza

# 'abril', 'mayo', 'septiembre', 'martes', 'julio'
# 'talca', 'paris', 'londres'

In [ ]:
# casos complejos

# 'hombre' + 'activista'
# 'mujer' + 'activista'
# 'hombre' - 'familia'
# 'mujer' - 'familia'

## Exploremos datos

Dos archivos `dataset_compras.tsv` y `clases.tsv`.

In [ ]:
!head dataset_compras.tsv

In [ ]:
!head clases.tsv

## Cargamos los datos de compras

Cargamos los datos en una estructura de la forma

```
compras = [ 
    (texto_compra_1, int_clase_compra_1),  
    (texto_compra_2, int_clase_compra_2),
    ...
    (texto_compra_N, int_clase_compra_N)
]

clases = [
    texto_clase_1,
    texto_clase_2,
    ...
]```

In [ ]:
compras = []
with open('dataset_compras.tsv') as f:
    for line in f:
        glosa, clase = line[:-1].split('\t')
        compras.append((glosa, int(clase)))
        
# ejemplos
compras[100:110]

In [ ]:
clases = []
with open('clases.tsv') as f:
    clases = [line[:-1] for line in f]
    
clases

## Vectores para frases y  similitud

Para convertir una lista de palabras en un vector, simplemente sumamos los vectores, y luego normalizamos el vector resultante:

In [ ]:
# si no hemos cargado los embeddings antes lo hacemos ahora
# we = KeyedVectors.load_word2vec_format('fasttext-sbwc.100k.vec', limit=100000)

In [ ]:
import numpy as np
from numpy.linalg import norm # para normalizar datos

In [ ]:
def to_vector(texto):
    tokens = texto.split()
    vec = np.zeros(300)
    for word in tokens:
        # si la palabra está la acumulamos
        if word in we:
            vec += we[word]
    return vec / norm(vec)

In [ ]:
texto = 'me gustan los gatos'
to_vector(texto)[:30]

Hacemos una función para similitud de textos, usando vectores y producto punto de numpy.

In [ ]:
def similarity(texto_1, texto_2):
    vec_1 = to_vector(texto_1)
    vec_2 = to_vector(texto_2)
    sim = vec_1 @ vec_2
    return sim

In [ ]:
texto_1 = 'no me gustan los gatos'
texto_2 = 'los felinos son lindos'
texto_3 = 'quiero comer pizza'

print(similarity(texto_1, texto_2))
print(similarity(texto_2, texto_3))
print(similarity(texto_1, texto_3))

In [ ]:
def clasifica(texto, clases):
    sims = [similarity(texto, clase) for clase in clases]
    indices = range(len(sims))
    ind_max = max(indices, key=lambda i: sims[i])
    return ind_max

In [ ]:
clase = clasifica("un cuarto de lomo liso", clases)
print(clases[clase])

In [ ]:
clase = clasifica("dos paracetamol para el dolor de cabeza", clases)
print(clases[clase])

In [ ]:
clase = clasifica("la cuenta del internet", clases)
print(clases[clase])

In [ ]:
clase = clasifica("whisky", clases)
print(clases[clase])

## Clasifica un dato al azar

In [ ]:
# elige una compra al azar (índice)
i = np.random.randint(len(compras))

# clase real
(compra, clase_real) = compras[i]
clase_pred = clasifica(compra, clases)

print('glosa compra:\t',compra)
print('clase predicha:\t', clases[clase_pred])
print('clase real:\t', clases[clase_real])

## Tarea: métricas de  acierto

Calcula la predicción para todos los texto y compara con la clase real. Usa `classification_report` y `accuracy_score`.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
%%time
# Hacemos la predicción para los primeros 5000 ejemplos

pred = []
real = []

for compra, clase_real in compras[:5000]:
    clase_pred = clasifica(compra, clases)
    pred.append(clase_pred)
    real.append(clase_real)

print(classification_report(real, pred))
print("Accuracy:", accuracy_score(real, pred))

## Vectorización

El anterios código tarda demasiado para clasificar todos los ejemplos. Para mejorar radicalmente el tiempo de ejecución podemos convertir todo en operación de matrices. Primero creamos una matriz con todos los vectores representantes de cada texto de compra, y luego una con los vectores de las clases.

In [ ]:
%%time

# pon todos los vectores de compras en una matriz
compras_vectores = [to_vector(texto) for texto, cls in compras]
X = np.vstack(compras_vectores)

# pon todos los vectores de clases en una matriz
clases_vectores = [to_vector(cls) for cls in clases]
C = np.vstack(clases_vectores)

print('X.shape =',X.shape)
print('C.shape =',C.shape)

Ahora podemos calcular la similitud con una simple multiplicación de matrices.

In [ ]:
%%time

#calcula las similitudes como un producto punto
similitudes = X @ C.T

pred = np.argmax(similitudes, axis=1)
real = [cls for _, cls in compras]

print(classification_report(real, pred))
print("Accuracy:", accuracy_score(real, pred))